In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import suncalc

import dask.dataframe as dd
from pathlib import Path
from tqdm import tqdm
import re

import datetime as dt

import matplotlib.pyplot as plt
from matplotlib import colors
import soundfile as sf
import matplotlib.patches as patches

In [2]:
import sys

sys.path.append("../src")
sys.path.append("../src/activity")

In [3]:
import subsampling as ss
import activity_assembly as actvt
import bout.assembly as bt
import comparison.data_assembly as comp
import comparison.plot as complot
from core import SITE_NAMES

from cli import get_file_paths
import plot
import pipeline

In [4]:
avail = np.arange(0, 720, 6) + 6
reset_3 = avail[np.where((3*60 % avail) == 0)[0]]
reset_4 = avail[np.where((4*60 % avail) == 0)[0]]
reset_6 = avail[np.where((6*60 % avail) == 0)[0]]
reset_12 = avail[np.where((12*60 % avail) == 0)[0]]
reset_24 = avail[np.where((24*60 % avail) == 0)[0]]

In [5]:
reset_3

array([  6,  12,  18,  30,  36,  60,  90, 180])

In [6]:
reset_4

array([  6,  12,  24,  30,  48,  60, 120, 240])

In [7]:
reset_6

array([  6,  12,  18,  24,  30,  36,  60,  72,  90, 120, 180, 360])

In [8]:
reset_12

array([  6,  12,  18,  24,  30,  36,  48,  60,  72,  90, 120, 144, 180,
       240, 360, 720])

In [9]:
reset_24

array([  6,  12,  18,  24,  30,  36,  48,  60,  72,  90,  96, 120, 144,
       180, 240, 288, 360, 480, 720])

In [5]:
f = 12
fact_ = reset_12[np.where((reset_12%f) == 0)[0]]
fact_

array([ 12,  24,  36,  48,  60,  72, 120, 144, 180, 240, 360, 720])

In [6]:
step = 1/12
step_by = np.arange(step, 1, step)
step_by

array([0.08333333, 0.16666667, 0.25      , 0.33333333, 0.41666667,
       0.5       , 0.58333333, 0.66666667, 0.75      , 0.83333333,
       0.91666667])

In [30]:
cycle_lengths = fact_
percent_ons = step_by
data_params = dict()
data_params["cycle_lengths"] = cycle_lengths
data_params["percent_ons"] = percent_ons
dc_tags = ss.get_list_of_dc_tags(cycle_lengths, percent_ons)

site_keys = ['Carp', 'Telephone', 'Central', 'Foliage']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'bout_time_percentage'
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, btp_arr = comp.generate_activity_btp_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 132/132 [22:12<00:00, 10.10s/it]


Carp HF


100%|██████████| 132/132 [05:04<00:00,  2.31s/it]


Telephone LF


...

100%|██████████| 132/132 [08:20<00:00,  3.79s/it]


Foliage LF


100%|██████████| 132/132 [10:38<00:00,  4.83s/it]


Foliage HF


100%|██████████| 132/132 [18:33<00:00,  8.43s/it]


In [7]:
cycle_lengths = fact_
percent_ons = step_by
data_params = dict()
data_params["cycle_lengths"] = cycle_lengths
data_params["percent_ons"] = percent_ons
dc_tags = ss.get_list_of_dc_tags(cycle_lengths, percent_ons)

site_keys = ['Carp', 'Telephone', 'Central', 'Foliage']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'call_rate'
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, callrate_arr = comp.generate_activity_call_rate_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 132/132 [05:27<00:00,  2.48s/it]


Carp HF


100%|██████████| 132/132 [01:12<00:00,  1.81it/s]


Telephone LF


...

100%|██████████| 132/132 [01:23<00:00,  1.58it/s]


Foliage LF


100%|██████████| 132/132 [01:31<00:00,  1.44it/s]


Foliage HF


100%|██████████| 132/132 [03:07<00:00,  1.42s/it]


In [7]:
cycle_lengths = fact_
percent_ons = step_by
data_params = dict()
data_params["cycle_lengths"] = cycle_lengths
data_params["percent_ons"] = percent_ons
dc_tags = ss.get_list_of_dc_tags(cycle_lengths, percent_ons)

site_keys = ['Carp', 'Telephone', 'Central', 'Foliage']
type_keys = ['LF', 'HF']
data_params["dc_tags"] = dc_tags
data_params['cur_dc_tag'] = '30of30'
data_params['metric_tag'] = 'activity_index'
data_params['index_time_block_in_secs'] = 5
data_params['recording_start'] = '00:00'
data_params['recording_end'] = '16:00'

for site_key in site_keys:
    for type_key in type_keys:
        print(site_key, type_key)
        data_params["site_tag"] = site_key
        data_params["site_name"] = SITE_NAMES[site_key]
        data_params["type_tag"] = type_key
        file_paths = get_file_paths(data_params)

        activity_arr, actvtind_arr = comp.generate_activity_index_percent_for_dc_schemes_and_cont(data_params, file_paths, save=True)

Carp LF


100%|██████████| 132/132 [05:19<00:00,  2.42s/it]


Carp HF


100%|██████████| 132/132 [01:11<00:00,  1.85it/s]


Telephone LF


...

100%|██████████| 132/132 [01:29<00:00,  1.48it/s]


Foliage LF


100%|██████████| 132/132 [02:03<00:00,  1.07it/s]


Foliage HF


100%|██████████| 132/132 [03:02<00:00,  1.38s/it]
